In [4]:
import pandas as pd
import numpy as np
import time
pd.set_option('display.max_colwidth', None) 

import tensorflow as tf
from transformers import CamembertTokenizer
from transformers import TFCamembertForSequenceClassification

from sklearn.base import BaseEstimator
from sklearn import metrics 

df = pd.read_pickle('/content/drive/MyDrive/Projets/allocine_dataset.pickle')
df_train = df['train_set']
df_val = df['val_set']
df_test = df['test_set']
df_test.head(1)

,film-url,review,polarity
0,http://www.allocine.fr/film/fichefilm-25385/critiques/spectateurs,"Magnifique épopée, une belle histoire, touchante avec des acteurs qui interprètent très bien leur rôles (Mel Gibson, Heath Ledger, Jason Isaacs...), le genre de film qui se savoure en famille! :)",1


Le Transfomer est un modèle de Deep Learning de type seq2seq qui a la particularité de n'utiliser que le mécanisme d'attention et aucun réseau récurrent ou convolutif. 

L'architecture du Transfomer a hérité du schéma encodeur-décodeur des réseaux récurrents. La partie "encodage" contient plusieurs encodeurs montés les uns à la suite des autres. La partie "décodage" est constituée de plusieurs décodeurs également montés les uns à la suite des autres mais prenant chacun, comme entrée supplémentaire, la sortie du 6ème encodeur.

L'entrée d'un encodeur est la sortie de l'encodeur précédent. L'entrée du premier encodeur est le vecteur d'embedding.

L'encodeur est constitué de deux blocs : 
- une couche "d'auto-attention"
- un réseau de neurones "Feed-forward" : son rôle est de préserver l'interdépendance des mots dans la représentation des séquences.

Le décodeur est également composé d'un bloc d'auto-attention et d'un Feed-forward mais il contient en plus une couche "Encoder-Decoder Attention" qui a pour but de permettre au décodeur de réaliser le mécanisme d'attention entre la séquence d'entrée (encodée) et la séquence de sortie (en cours de décodage).

L'idée du concept d'attention est de mesurer dans quelle mesure deux éléments de deux séquences sont liés. Dans un contexte de séquence à séquence en PNL, le mécanisme d'attention visera à indiquer au reste du modèle les mots de la séquence B auxquels il faut prêter le plus d'attention lors du traitement d'un mot de la séquence A.

BERT est transformer pré-entraînés sur des millions de textes et qu’il est possible de fine-tuner pour des tâches spécifiques. Ces modèles, aujourd’hui, sont ceux qui permettent d’avoir les meilleurs résultats pour les tâches classiques : classification de texte ou NER. 

## Preprocessing

Pour ce faire, nous utilisons CamembertTokenizer, un tokenizer fourni par Huggingface.

In [5]:
model_name = "camembert-base"
tokenizer = CamembertTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/508 [00:00<?, ?B/s]

Testons le bon fonctionnement du tokenizer.

In [7]:
tokenizer.encode(df['test_set']['review'][0])[:10]


[5, 12158, 21, 21323, 7, 28, 455, 738, 7, 8353]

In [8]:
tokenizer.decode(tokenizer.encode(df['test_set']['review'][0])[:10])

'Magnifique épopée, une belle histoire, touchant'

Le mécanisme d'attention propre au transformer nécessite un encodage particulier des reviews.

In [9]:
def encode_reviews(tokenizer, reviews, max_length):
    
    token_ids = np.zeros(shape=(len(reviews), max_length), dtype=np.int32)
    
    for i, review in enumerate(reviews):
        encoded = tokenizer.encode(review, max_length=max_length)
        token_ids[i, 0:len(encoded)] = encoded
    
    attention_mask = (token_ids != 0).astype(np.int32)
    
    return {"input_ids": token_ids, "attention_mask": attention_mask}

On réalise ensuite le padding des reviews selon un certain threshold (se basant sur de la distribution des longueurs moyennes des reviews, comme précédemment)

In [ ]:
# reviews_len = [len(tokenizer.encode(review, max_length=512)) for review in df['train_set']['review']]
# reviews_len = pd.Series(reviews_len)
# reviews_len.describe(percentiles=[.25, .50, .75, 0.95, 0.99, 0.999])

In [ ]:
MAX_LEN = 350

encoded_train = encode_reviews(tokenizer, df['train_set']['review'], MAX_LEN)
encoded_valid = encode_reviews(tokenizer, df['val_set']['review'], MAX_LEN)
encoded_test = encode_reviews(tokenizer, df['test_set']['review'], MAX_LEN)

In [ ]:
y_train = np.array(df_train['polarity'])
y_val = np.array(df_val['polarity'])
y_test = np.array(df_test['polarity'])

## Modelling

Pour ce faire, nous utilisons TFCamembertForSequenceClassification, un modèle français pré-entraîné fourni par Huggingface. Par nature, les transformateurs sont généralistes et peuvent fournir d'excellents résultats même sans réentraînement. 

Cela dit, nous obtenons généralement des résultats encore meilleurs en ajustant le modèle sur le corpus d'intérêt (ici nos critiques de films en français). 

In [ ]:
# pretrained model
model = TFCamembertForSequenceClassification.from_pretrained("jplu/tf-camembert-base")
   

Downloading:   0%|          | 0.00/508 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/520M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFCamembertForSequenceClassification.

Some layers of TFCamembertForSequenceClassification were not initialized from the model checkpoint at jplu/tf-camembert-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5, epsilon=1e-07)
loss_func = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) 

model.compile(optimizer=optimizer, 
              loss=loss_func, 
              metrics=['accuracy'])

In [ ]:
# history = model.fit(
#     encoded_train, y_train, epochs=1, batch_size=4, 
#     validation_data=(encoded_valid, y_val), verbose=1
# )

Cet entraînement prend beaucoup de temps (en utilisant l'ensemble des données) : 1 epoch == 4h tesla k80.

### Entraînement plus rapide sur un échantillon

Ici, nous allons réduire considérablement le temps de formation en utilisant la technique du early stopping. Nous limiterons également l'apprentissage à un échantillon de l'ensemble de données. Dans un contexte de production, nous utiliserions l'ensemble des données. Même dans le cas du réglage fin d'un modèle, comme souvent en apprentissage profond, plus il y a de données, mieux c'est. 

In [ ]:
# Building a model with early stopping policy 
# BaseEstimator class allows to implement an estimator in a sklearn fashion
class CustomizedCamembert(BaseEstimator):
    def __init__(self, transformers_model, max_epoches, batch_size, validation_data):
        self.model = transformers_model
        self.max_epoches = max_epoches
        self.batch_size = batch_size
        self.validation_data = validation_data
        
    def fit(self, X, y):
        early_stopping = tf.keras.callbacks.CustomizedCamembert(
            monitor='val_loss', mode='auto', patience=2, verbose=1, restore_best_weights=True)        
        # trains model
        self.model.fit(
            X, y, validation_data=self.validation_data,
            epochs=self.max_epoches, batch_size=self.batch_size,
            callbacks=[early_stopping], verbose=1)        
        return self
    
    def predict(self, X):        
        scores = self.model.predict(X)
        # get highest proba from softmax fn
        y_pred = np.argmax(scores[0], axis=1)
        return y_pred

Ici on définit la fonction d'entraînement faisant appel au modèle précédemment définit.

In [ ]:
# here we define the whole training loop fn
def training_loop(camembert_model, initial_weights, sizes,
                        enc_train_reviews, train_labels,
                        enc_val_reviews, val_labels,
                        enc_test_reviews, test_labels):

    test_accuracies = []
    for size in sizes:        
        enc_train_reviews = enc_train_reviews[:size]
        y_train = train_labels[:size]
        enc_val_reviews = enc_val_reviews[:size]
        y_val = val_labels[:size]
        enc_test_reviews = enc_test_reviews[:size]
        y_test = test_labels[:size]
        
        # Reset weights to initial value
        # Putting a super low epoch numbers cause of time constraints
        # In a production setting, set a higher number (10/20)
        camembert_model.set_weights(initial_weights)
        best_model = CustomizedCamembert(
            camembert_model, max_epoches=2, batch_size=4,
            validation_data=(enc_val_reviews, y_val))
        
        # trains model
        best_model.fit(enc_train_reviews, y_train)
        
        # tests model
        y_pred = best_model.predict(enc_test_reviews)
        test_acc = metrics.accuracy_score(y_test, y_pred)
        test_accuracies.append(test_acc)
        print("Test accuracy: " + str(test_acc))
        
    return test_accuracies    


In [ ]:
init_weights = model.get_weights()
model.summary()

Model: "tf_camembert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 roberta (TFRobertaMainLayer  multiple                 110031360 
 )                                                               
                                                                 
 classifier (TFRobertaClassi  multiple                 592130    
 ficationHead)                                                   
                                                                 
Total params: 110,623,490
Trainable params: 110,623,490
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# the more data the better, but because of time constraints, using a sample
# after multiple tries, 10k training reviews already bring a good acc

samples = [10000] # [1000, 10000, 50000, 100000]
test_acc = training_loop(model, init_weights, samples, 
                         encoded_train, y_train,
                         encoded_valid, y_val, 
                         encoded_test, y_test)

In [ ]:
# saving the weight of the best finetuned model 
model.save_weights('/content/drive/MyDrive/Projets/camembert_weights.hdf5')

## Testing the best model

In [ ]:
model.load_weights('/content/drive/MyDrive/Projets/camembert_weights.hdf5')

scores = model.predict(encode_reviews(tokenizer, df['test_set']['review'], MAX_LEN))
y_pred = np.argmax(scores[0], axis=1)

print(metrics.classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.98      0.94      0.96     10408
           1       0.94      0.98      0.96      9592

    accuracy                           0.96     20000
   macro avg       0.96      0.96      0.96     20000
weighted avg       0.96      0.96      0.96     20000



In [ ]:
encoded_test = encode_reviews(tokenizer, df['test_set']['review'], MAX_LEN)

times = []
for i in range(500):
    x = {'input_ids': np.array([encoded_test['input_ids'][i], ]),
    'attention_mask':  np.array([encoded_test['attention_mask'][i], ])}
    t0 = time.time()
    y_pred = model.predict(x)
    t1 = time.time()
    times.append(t1 - t0)    

In [ ]:
np.mean(times)

0.13456082940101624

Ici, il y a un gain en accuracy assez net par rapport aux modèles de type RNN ou même aux word embeddings plus basiques. 

Mais ce gain se fait au prix d'un temps d'inférence en moyenne 3 fois supérieur. 

Suivant le nombre d'appels à l'API, les besoins en accuracy ou même notre budget d'exploitation, il faudra réaliser un arbitrage. Ici, il s'agit d'un projet à but pédagogique. Nous allons donc implémenter le modèle le plus performant : Camembert.